# Projeto 13: Rede Neural Convolucional MNIST Aumento de Imagens

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 4.0MB 50.1MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets
import torchvision.transforms as transforms

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
transform_train = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(degrees=7,
                                                              translate=(0,0.07),
                                                              shear=7,
                                                              scale=(1,1.2)),
                                      transforms.ToTensor()
])

In [5]:
transform_test = transforms.ToTensor()

In [6]:
train = datasets.MNIST(root='.', train=True, download=True, transform=transform_train)
test = datasets.MNIST(root='.', train=False, download=True, transform=transform_test)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [7]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128)
test_loader = torch.utils.data.DataLoader(test, batch_size=128)

## Etapa 3: Construção do modelo

In [8]:
class cls_torch(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
    self.activation = nn.ReLU()
    self.pool = nn.MaxPool2d(kernel_size=(2,2))
    self.flatten = nn.Flatten()
    self.linear = nn.Linear(in_features=32*13*13, out_features=128)
    self.output = nn.Linear(128,10)

  def forward(self, X):
    X = self.pool(self.activation(self.conv(X)))
    X = self.flatten(X)
    X = self.activation(self.linear(X))
    X = self.output(X)
    return X

In [9]:
net = cls_torch()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [13]:
net.to(device)

cls_torch(
  (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (activation): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten()
  (linear): Linear(in_features=5408, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

## Etapa 4: Treinamento do modelo

In [18]:
def training_loop(loader, epoch):

  running_loss = 0.
  running_acc = 0.

  for i, data in enumerate(loader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = net(inputs)

    loss = criterion(outputs, labels)
    loss.backward()

    optimizer.step()

    running_loss += loss.item()

    ps = F.softmax(outputs)
    top_p, top_class = ps.topk(k=1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    acc = torch.mean(equals.type(torch.float))

    running_acc += acc

    #print('\rÉpoca {:3d} - Loop {:3d} de {:3d}: perda {:03.2f} - precisão {:03.2f}' \
    #      .format(epoch + 1, i + 1, len(loader), loss, acc))

  print('\rÉPOCA {:3d} FINALIZADA: Perda {:.5f} - Precisão {:.5f}' \
        .format(epoch + 1, running_loss/len(loader), running_acc/len(loader)))

In [19]:
for epoch in range(5):
  print('\nTREINANDO...')
  training_loop(train_loader, epoch)

  net.eval()
  print('\nVALIDANDO...')
  training_loop(test_loader, epoch)

  net.train()


TREINANDO...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


ÉPOCA   1 FINALIZADA: Perda 0.09940 - Precisão 0.96927

VALIDANDO...
ÉPOCA   1 FINALIZADA: Perda 0.02603 - Precisão 0.99239

TREINANDO...
ÉPOCA   2 FINALIZADA: Perda 0.09376 - Precisão 0.97161

VALIDANDO...
ÉPOCA   2 FINALIZADA: Perda 0.02267 - Precisão 0.99357

TREINANDO...
ÉPOCA   3 FINALIZADA: Perda 0.08863 - Precisão 0.97280

VALIDANDO...
ÉPOCA   3 FINALIZADA: Perda 0.01945 - Precisão 0.99614

TREINANDO...
ÉPOCA   4 FINALIZADA: Perda 0.08375 - Precisão 0.97366

VALIDANDO...
ÉPOCA   4 FINALIZADA: Perda 0.01707 - Precisão 0.99644

TREINANDO...
ÉPOCA   5 FINALIZADA: Perda 0.07955 - Precisão 0.97487

VALIDANDO...
ÉPOCA   5 FINALIZADA: Perda 0.01499 - Precisão 0.99713
